In [1]:
!pip install --quiet arcgis

In [2]:
from arcgis.gis import GIS;
from arcgis.features import FeatureLayer,Table;
import pandas as pd;
import requests;


In [3]:
ag_url    = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/ATTAINS_Geo/ATTAINS_Assessments/MapServer';
ag_points = FeatureLayer(ag_url + "/0");
ag_lines  = FeatureLayer(ag_url + "/1");
ag_areas  = FeatureLayer(ag_url + "/2");
ag_cats   = FeatureLayer(ag_url + "/3");
ag_attr   = Table(ag_url + "/4");


In [36]:
base_tbl = [
  ['AL'],['AK'],['AS'],['AZ'],['AR'],['CA'],['CO'],['CT'],['DC'],['DE'],['FL'],['GA'],['GU'],['HI'],['ID'],['IL'],
  ['IN'],['IA'],['KS'],['KY'],['LA'],['ME'],['MD'],['MA'],['MI'],['MN'],['MP'],['MS'],['MO'],['MT'],['NE'],['NV'],
  ['NH'],['NJ'],['NM'],['NY'],['NC'],['ND'],['OH'],['OK'],['OR'],['PA'],['PR'],['RI'],['SC'],['SD'],['TN'],['TX'],
  ['UT'],['VI'],['VT'],['VA'],['WA'],['WV'],['WI'],['WY']
] 
  
df = pd.DataFrame(base_tbl,columns = ['state']);
df = df.set_index('state')


In [37]:
cnt_points = ag_points.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle'
);

cnt_lines = ag_lines.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle'
);

cnt_areas = ag_areas.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle'
);

cnt_attr = ag_attr.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle'
);

cnt_cats = ag_cats.query(
    return_all_records = False
   ,group_by_fields_for_statistics = 'state,reportingcycle'
   ,out_statistics = [{"statisticType":"count","onStatisticField":"state,reportingcycle","outStatisticFieldName":"count"}]
   ,order_by_fields = 'state,reportingcycle'
);


In [38]:
df2 = df.join(
    other = cnt_cats.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"reportingcycle":"cats_cycle","count":"cats_count"}).drop('OBJECTID',1);

df2 = df2.join(
    other = cnt_attr.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"reportingcycle":"attr_cycle","count":"attr_count"}).drop('OBJECTID',1);

df2 = df2.join(
    other = cnt_points.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"point_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.join(
    other = cnt_lines.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"line_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.join(
    other = cnt_areas.sdf.set_index('state')
   ,how = 'left'
);
df2 = df2.rename(columns={"count":"area_count"}).drop('OBJECTID',1).drop('reportingcycle',1);

df2 = df2.where((pd.notnull(df2)), None);


In [39]:
df2


,cats_cycle,cats_count,attr_cycle,attr_count,point_count,line_count,area_count
state,,,,,,,
AL,2018,23005,2018,1810,None,1776,307
AK,None,None,2016,63,None,41,22
AS,None,None,None,None,None,None,None
AZ,2018,3709,2018,70,None,54,16
AR,None,None,None,None,None,None,None
CA,2016,37596,2016,1797,None,1454,343
CO,2018,63848,2018,1482,None,1161,325
CT,2016,4664,2016,1444,None,1051,393
DC,2018,101,2018,36,None,25,11
